In [1]:
import networkx as nx
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import pickle
import networkx as nx
import matplotlib as mlp
# mlp.use("Qt5Agg")
import matplotlib.pyplot as plt
import community
import scipy

# Functions

### Function to threshold graphs

In [2]:
def threshold(G, corr_direction):
    ##Creates a copy of the graph
    H = G.copy()
    for stock1, stock2, weight in list(G.edges(data=True)):
        if corr_direction == "positive":
            if weight["weight"] <0:
                H.remove_edge(stock1, stock2)
        else:
            if weight["weight"] >=0:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to unpickle data

In [3]:
def onetoughjar(path2dic):
    with open(path2dic, 'rb') as pickle_file:
        try:
            while True:
                output = pickle.load(pickle_file)
        except EOFError:
            pass
    return(output)

### Function to create graphs of all the nodes

In [4]:
def grace_graph(graph, group):
    G=graph
    positions = nx.circular_layout(G)
    size = 100
    title= "Modularity and edge weights \n of average %s graph"%(group)
    save="%s_graph.png"%(group)
    
    edges,weights = zip(*nx.get_edge_attributes(G, 'weight').items())

    nodes, color = zip(*nx.get_node_attributes(G, 'modules').items()) #if your modules are named different change here
    # nodes, names = zip(*nx.get_node_attributes(graph, 'label').items()) #if your modules are named different change here
    
    #Figure size
    plt.figure(figsize=(80,50))

    #draws nodes
    color = np.array(color)
    n_color=len(list(set(color)))
    # nColormap=plt.cm.Set3 #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    # get discrete colormap
    nColormap = plt.get_cmap('Set3', n_color)

    # scaling
    sz=np.array(size)
    scale=15000/sz.max()
    sza=sz*scale
    # print(sz.shape)

    y=nx.draw_networkx_nodes(G,positions,
                           node_color=color,
                           node_size=sza,
                           alpha=0.8,
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(G, positions,
                            # labels = label_dict,
                            font_size=50,
                            font_family='sans-serif',
                            fontweight = 'bold')

    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    # scaling
    wt=list(set(weights))
    wt=np.array(wt)
    wt2=-np.sort(-wt)
    wt0=wt2[1]

    escale=1/wt0
    esza=weights*escale
    E=list(set(esza))
    E2=-np.sort(-np.array(E))
    M=1
    m=0

    x=nx.draw_networkx_edges(G, positions,
                           edge_list=edges,
                           style='solid',
                           width = np.square(esza)*20,
                           edge_color = esza,
                           edge_cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)

    #COLORBAR STUFF
    node_bar=plt.colorbar(y, label='Module value')

    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))


    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = m, vmax=M))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
        l.set_verticalalignment('center')

    node_bar.set_label('Modularity',fontsize = 50)
    edge_bar.set_label('Strength of edge weight',fontsize = 50)
    # Final plot stuff
    plt.axis('off')

    plt.title(title, fontsize = 100)
    basepath='/Users/gracer/Google Drive/HCP_graph/1200/images'

    plt.savefig(os.path.join(basepath,save), format="PNG")
    plt.show()
    return()

### Function to create communities

In [5]:
def com_create(mu):
    partition = community.best_partition(mu)
    vals = list(partition.values())
    nx.set_node_attributes(mu, partition, 'modules')
    return((partition,vals, mu))

### Function to generate p values

In [6]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

### Function to visualize the module graphs

In [7]:
def inmod_graph(graph, group):
#     graph=threshold(graph,dirp,minp)
    e,w = zip(*nx.get_edge_attributes(graph, 'weight').items())
    positions = nx.circular_layout(graph)
    size = 100
    title= "Modularity and edge weights \n of average %s graph"%(group)
    save="%s_graph.png"%(group)
    
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())

    nodes, color = zip(*nx.get_node_attributes(graph, 'modules').items()) #if your modules are named different change here
    # nodes, names = zip(*nx.get_node_attributes(graph, 'label').items()) #if your modules are named different change here
    g=graph
    #Figure size
    plt.figure(figsize=(80,50))

    #draws nodes
    color = np.array(color)
    n_color=len(list(set(color)))
    # nColormap=plt.cm.Set3 #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    # get discrete colormap
    nColormap = plt.get_cmap('Set3', n_color)

    # scaling
    sz=np.array(size)
    scale=15000/sz.max()
    sza=sz*scale
    # print(sz.shape)

    y=nx.draw_networkx_nodes(g,positions,
                           node_color=color,
                           node_size=sza,
                           alpha=0.8,
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(g, positions,
                            # labels = label_dict,
                            font_size=50,
                            font_family='sans-serif',
                            fontweight = 'bold')

    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
#     # scaling
#     wt=list(set(weights))
#     wt=np.array(wt)
#     wt2=-np.sort(-wt)
#     wt0=wt2[1]

#     escale=1/wt0
#     esza=weights*escale
#     E=list(set(esza))
#     E2=-np.sort(-np.array(E))
    M=1
    m=-1

    x=nx.draw_networkx_edges(g, positions,
                           edge_list=edges,
                           style='solid',
                           width = np.square(weights)*100,
                           edge_color = weights,
                           edge_cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)

    #COLORBAR STUFF
    node_bar=plt.colorbar(y, label='Module value')

    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))


    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = m, vmax=M))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
        l.set_verticalalignment('center')

    node_bar.set_label('Modularity',fontsize = 50)
    edge_bar.set_label('Strength of edge weight',fontsize = 50)
    # Final plot stuff
    plt.axis('off')

    plt.title(title, fontsize = 100)
    basepath='/Users/gracer/Google Drive/HCP_graph/1200/images'

    plt.savefig(os.path.join(basepath,save), format="PNG")
    plt.show()
    return()

In [8]:
def module_fig(G, group, IC):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())

    positions=nx.circular_layout(G)
    plt.figure(figsize=(80,50))
    ### NODES ####
    color = np.array(list(G.nodes))
    color = np.array(color)
    n_color=len(list(set(color)))
    nColormap = plt.get_cmap('Set3', n_color)

    y=nx.draw_networkx_nodes(G,positions,
                           node_color=color,
                           node_size=15000,
                           alpha=1.0,
                           cmap= nColormap,
                           vmin=0,vmax=n_color )

    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=100,
                            font_family='sans-serif', fontweight = 'bold')
    
    ### EDGES ####
    weights=np.array(weights)

    m=weights.min()
    M=weights.max()
    
    eColormap=plt.cm.gist_rainbow
    
    x=nx.draw_networkx_edges(G, positions, 
                             edge_list=edges,
                             style='solid', 
                             width = weights,
                             edge_color = weights, 
                             edge_vmin=m, 
                             edge_vmax=M, 
                             edge_cmap= eColormap)

    
    node_bar=plt.colorbar(y, label='Module value')
    
    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))


    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = m, vmax=M))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(50)
        l.set_verticalalignment('center')

    node_bar.set_label('Modularity',fontsize = 50)
    edge_bar.set_label('Strength of edge weight',fontsize = 50)
    
    plt.title("Module Connectivity Weights %s %s"%(group, IC), fontsize = 100)
    plt.axis('off')
    basepath='/Users/gracer/Google Drive/HCP_graph/1200/images'
#     plt.savefig(os.path.join(basepath,"modularity_%s.png"%(Type)), format="PNG")
    # plt.show()

### Function to look at the p values quickly

In [9]:
def skimP(mat):
    mask = np.zeros_like(mat)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=(14, 10))
        ax = sns.heatmap(mat, mask=mask,annot=True, center=0.05,square=True, cmap='gist_ncar')
#         ax.set_title(name)

### Function to generate graph and metrics

In [10]:
def mu_make_graphs(key, values, direction):
    ########################################
    cor_matrix = np.asmatrix(values)
    G = nx.from_numpy_matrix(cor_matrix)
    ########################################
    tG = threshold(G, direction)
    if len(list(tG.edges(data=True))) > 0:
        if direction == 'negative':
            print('start negative')
            edges,weights = zip(*nx.get_edge_attributes(tG,'weight').items())
            wts=tuple(abs(np.array(weights)))
            total=[]
            for x in enumerate(edges):
                y={'weight':wts[x[0]]}
                total.append(x[1]+(y,))
            tG.add_edges_from(total)

        ########################################
        (partition,vals,graph)=com_create(tG)
        nx.set_node_attributes(tG, partition, 'modules')
        ########################################
        x=abs(cor_matrix)
        mu=x.mean()
        ########################################
        clustering = nx.clustering(tG, weight=True)
        ########################################
        centrality = nx.betweenness_centrality(tG, weight=True)
        ########################################
        nx.set_node_attributes(tG, centrality, 'centrality')
        nx.set_node_attributes(tG, clustering, 'clustering')
        nx.set_node_attributes(tG, partition, 'modules')

        stocks = values.index.values
        tG = nx.relabel_nodes(tG,lambda x: stocks[x])
        ########################################
        return({'mean_FC':mu, 'graphs':tG, 'clustering_coeff':clustering, 'btn_centrality':centrality,  'modules':{'partition':partition,
    'values':vals,'graph':graph}})
    else:
        print('this is empty')
    

## Base dictionary structure

In [124]:
base_dict={'late':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           },'early':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           }
          }

## Load in appropriate files
1. Header = These are the ICs from the 15 dimenstion
2. labels = These are labels from the glasser and gordon merged brains (allowing for cortex and subcortex)
3. covars = These are the values of BMI, age at onset of menses, race, age, and head motion. The same file was used in FSLNETS

In [12]:
header=['IC1','IC2','IC3','IC4','IC5','IC6','IC7','IC8','IC9','IC10','IC11','IC12','IC13','IC14','IC15']

In [13]:
labels = pd.read_csv('/Users/gracer/Google Drive/HCP_graph/1200/brains/lables_glasser.csv', sep=',')

In [89]:
covars=pd.read_csv('/Users/gracer/Google Drive/HCP_graph/1200/datasets/subs510.csv', sep=',', header=None)

In [96]:
covars.columns=['idx','subject','group','BMI','AoM','int','race','age','motion']

In [97]:
covars.drop_duplicates(subset='subject', keep="last", inplace=True)

In [98]:
covars.shape

(510, 9)

In [99]:
covars['subject']=covars['subject'].astype('int64')

In [113]:
high_low=pd.read_csv('/Users/gracer/Google Drive/HCP_graph/1200/datasets/post_hoc.csv', sep=',')

In [114]:
hl=high_low[['Subject','AoM']]
hl=hl.dropna()
hl.columns = ["subject", "high_low"]

### Load in all timeseries data from each participant into data_dict. 
These are the netmat2 files. They are zscores per TR

In [120]:
raw_dict={'late':hl.loc[hl['high_low']=='late'],'early': hl.loc[hl['high_low'] == 'early']}
data_dict={'late':{},'early':{}}
for key, df in raw_dict.items():
    print(key)
    for sub in df['subject']:
        path=os.path.join(
            '/Users/gracer/Downloads/HCP_S1200_PTNmaps_d15_25_50_100/3T_HCP1200_MSMAll_d15_ts2_Z/puberty/%i.txt'%sub)
        data_dict[key][sub]=pd.read_csv(path,sep='\t',names=header)

late
early


In [122]:
len(list(data_dict['early'].keys()))

87

Check that you have 212 subjects/files

## Subset the ICs of interest
Don't need all the ICs. Just the ones already associated with BMI and puberty. This allows us to look at the relationships within IC.

In [126]:
interest

{'late': {'BMI': {'IC1': {},
   'IC3': {},
   'IC4': {},
   'IC5': {},
   'IC6': {},
   'IC7': {},
   'IC8': {},
   'IC9': {},
   'IC10': {},
   'IC11': {},
   'IC12': {},
   'IC13': {},
   'IC14': {},
   'IC15': {}},
  'AoM': {'IC12': {}, 'IC15': {}},
  'int': {'IC2': {}, 'IC3': {}, 'IC6': {}, 'IC12': {}, 'IC13': {}}},
 'early': {'BMI': {'IC1': {},
   'IC3': {},
   'IC4': {},
   'IC5': {},
   'IC6': {},
   'IC7': {},
   'IC8': {},
   'IC9': {},
   'IC10': {},
   'IC11': {},
   'IC12': {},
   'IC13': {},
   'IC14': {},
   'IC15': {}},
  'AoM': {'IC12': {}, 'IC15': {}},
  'int': {'IC2': {}, 'IC3': {}, 'IC6': {}, 'IC12': {}, 'IC13': {}}}}

In [128]:
interest={'late':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           },'early':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           }
          }
for t, dt in interest.items():
    print(t)
    for x,y in dt.items():
        print(x)
        for k,v in y.items():
            print(k)
            for key, value in data_dict[t].items():
                v[key]=value[k]    

late
BMI
IC1
IC3
IC4
IC5
IC6
IC7
IC8
IC9
IC10
IC11
IC12
IC13
IC14
IC15
AoM
IC12
IC15
int
IC2
IC3
IC6
IC12
IC13
early
BMI
IC1
IC3
IC4
IC5
IC6
IC7
IC8
IC9
IC10
IC11
IC12
IC13
IC14
IC15
AoM
IC12
IC15
int
IC2
IC3
IC6
IC12
IC13


## Create datatables with the rows as subjects and columns as parcels per IC

From here on we will seperate all ICs from each other. This allows us to look at the parcellation per IC. The dfs is a dictionary of all ICs of interest. The columns are the parcels and rows are the subjects.

In [129]:
dfs = {'late':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           },'early':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           }
          }
for t, df in interest.items():
    for group, value in df.items():
        for IC,v in value.items():
            x=pd.DataFrame.from_dict(v, orient='index')
            print(x.shape)
            x['subject']=x.index
            x['subject'] = x['subject'].astype('int64')
            hl['subject']= hl['subject'].astype('int64')
            y=pd.merge(x, covars, left_on='subject', right_on='subject')
            print(y.shape)
            z=pd.merge(y, hl, left_on='subject', right_on='subject')
            print(z.shape)
            del z['group']
            z.set_index('subject', inplace=True)
            dfs[t][group][IC]=z

(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(125, 379)
(125, 388)
(125, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 379)
(87, 388)
(87, 389)
(87, 37

## Thresholding the zscores 
Only keeping zscores > 3.5 and greater than 75 of the participants must have a value for the score to keep the column. This is to threshold out noise and missing data

In [131]:
zscores_thr= {'late':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           },'early':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           }
          }

for t, df in dfs.items():
    for group, value in df.items():
        print(group)
        for IC,v in value.items():
            print(IC)
            x = v[(v.iloc[:,1:379]>3.5) | (v.iloc[:,1:379]<-3.5)]
            y=x.dropna(axis=1, how='all')
            test=y.isna().mean()
            q = y.loc[:, y.isna().mean() < .25]
            z=pd.concat([v.iloc[:,379:], q], axis=1)
            zscores_thr[t][group][IC]=z

BMI
IC1
IC3
IC4
IC5
IC6
IC7
IC8
IC9
IC10
IC11
IC12
IC13
IC14
IC15
AoM
IC12
IC15
int
IC2
IC3
IC6
IC12
IC13
BMI
IC1
IC3
IC4
IC5
IC6
IC7
IC8
IC9
IC10
IC11
IC12
IC13
IC14
IC15
AoM
IC12
IC15
int
IC2
IC3
IC6
IC12
IC13


In [133]:
zscores_thr['early']['BMI']['IC1']

,idx,BMI,AoM,int,race,age,motion,high_low,20,21,...,214,217,218,219,221,314,315,350,354,357
subject,,,,,,,,,,,,,,,,,,,,,
103818,32,-1.388836,-1.681529,2.335367,1,32,0.092165,early,7.10279,8.63813,...,NaN,4.09880,9.29575,7.08002,8.33374,-5.99679,-6.08389,8.71839,9.04678,NaN
107321,72,-2.768836,-2.681529,7.424712,2,25,0.041022,early,8.09248,6.99411,...,11.59180,6.31406,9.08030,6.39336,10.31090,-4.38226,-5.79642,10.19960,7.48249,12.89170
110007,92,5.711164,-3.681529,-21.025815,2,33,0.106553,early,10.66330,13.03760,...,8.36237,4.09352,6.06463,7.04072,9.00264,NaN,-3.86713,13.13160,9.87230,13.36000
111009,96,4.391164,-1.681529,-7.383869,1,27,0.107450,early,5.41019,8.80593,...,5.42408,4.82975,6.26391,4.07961,4.77211,-5.54656,-5.38236,7.07169,7.65790,5.81248
111312,104,1.721164,-1.681529,-2.894187,1,31,0.077252,early,9.55988,10.39110,...,9.72149,10.26560,9.87100,8.23288,10.45330,-5.34339,NaN,12.20910,8.94239,10.01950
112314,116,6.641164,-1.681529,-11.167308,5,30,0.058293,early,NaN,6.59585,...,5.76926,NaN,NaN,NaN,3.53372,-7.94338,-9.09349,4.55243,NaN,NaN
114318,136,-3.138836,-3.681529,11.555714,6,22,0.064025,early,6.84018,12.96430,...,16.19350,13.13680,6.88480,7.26034,6.55819,-11.03030,-7.71379,7.74241,7.03511,4.57389
115017,144,6.741164,-1.681529,-11.335461,10,32,0.114451,early,8.22373,8.49844,...,4.50086,NaN,7.75583,4.89631,10.82630,-3.89816,-3.98929,3.60894,8.51923,7.17801
117021,156,-1.388836,-1.681529,2.335367,1,30,0.062437,early,6.33682,5.65532,...,11.43420,5.76365,4.36603,7.18344,4.79309,-7.69384,-7.07146,8.15827,7.56204,NaN


# Generating the graphs with positive values

## Important
### Correlations
This is probably the most important part. Here we are taking our (subxparcel) dataframes we thresholded above and we are generating a correlation matrix using a Pearson's Product correlation (R). Within each matrix, there will be positive and negative correlation values. This can be a problem with the graph metrics we want to use.
### Graphs
To avoid the issue of negative and positive values, we are going to threshold each correlation matrix twice. 
1. Positive correlations
2. Negative correltions

We will then use the absolute value of the negative correlations to mimic a positive correlation. We will keep these seperate for the rest of the analysis.

In [136]:
graphs = {'late':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}
           },'early':
           {'BMI':
            {'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':
            {'IC12':{},'IC15':{}},
            'int':
            {'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}}}

for t, item in zscores_thr.items():
    for group, value in item.items():
        print(key)
        for IC, v in value.items():
            print(group)
            graphs[t][group][IC]=mu_make_graphs(IC, v.corr(), 'positive')


979984
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
979984
AoM
AoM
979984
int
int
int
int
int
979984
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
BMI
979984
AoM
AoM
979984
int
int
int
int
int


## Generating graphs with negative values
Unable to get communities with negative values need to keep seperate

In [ ]:
nega_graphs={'BMI':{'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':{'IC12':{},'IC15':{}},
            'int':{'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}}

for group, value in zscores_thr.items():
    print(group)
    for IC, v in value.items():
        print(IC)
        nega_graphs[group][IC]=mu_make_graphs(IC, v.corr(), 'negative')
        

## Visualizing the total graphs for all ICs in the interaction group

These graphs will show all the nodes in each IC (positive and negative). It will also show what community each node belongs to. Community struture will be a measure of density. Denser connections will form modules. 

In [ ]:
for key, value in graphs['int'].items():
    grace_graph(value['graphs'], 'Int_%s'%key)

In [ ]:
for key, value in nega_graphs['int'].items():
    grace_graph(value['graphs'], 'Int_%s'%key)

## writing all the adjacency matrices and topologocial values to csv to save

In [ ]:
for key, value in graphs.items():
    print(key)
    for k, v in value.items():
        print(k)
        cc=pd.DataFrame.from_dict(v['clustering_coeff'], orient='index')
        btw=pd.DataFrame.from_dict(v['btn_centrality'], orient='index')
        mod=pd.DataFrame.from_dict(v['modules']['partition'], orient='index')
        test3 = pd.concat([cc, btw, mod], axis=1)
        test3.columns = ['cc','btw','mod']
        print(test3.shape)
        test3.set_index([list(zscores_thr[key][k].columns.values)], inplace=True)
        adj=nx.to_pandas_adjacency(v['graphs'])
        print(adj.shape)
        df=adj.merge(test3, left_index=True, right_index=True)
        pd.DataFrame.to_csv(df, '/Users/gracer/Google Drive/HCP_graph/1200/datasets/%s_%s_thresh.csv'%(key,k))

In [ ]:
for key, value in nega_graphs.items():
    print(key)
    for k, v in value.items():
        print(k)
        cc=pd.DataFrame.from_dict(v['clustering_coeff'], orient='index')
        btw=pd.DataFrame.from_dict(v['btn_centrality'], orient='index')
        test3 = pd.concat([cc, btw], axis=1)
        test3.columns = ['cc','btw']
        test3.set_index([list(zscores_thr[key][k].columns.values)], inplace=True)
        pd.DataFrame.to_csv(test3, '/Users/gracer/Google Drive/HCP_graph/1200/datasets/%s_%s_thresh_neg.csv'%(key,k))

## Visualizing the within module graph 
These are graphs that look at the connectivity within each module. This shows us how the most related areas (densely connected) interact with each other. 

In [ ]:
subgraphs={'BMI':{'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':{'IC12':{},'IC15':{}},
            'int':{'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}}


for IC, value in graphs['int'].items():
    print(IC)
    G=value['graphs']
    mod=G.nodes(data=True)['int']['modules']
    print(mod)
    interest=[k for k,v in nx.get_node_attributes(G, 'modules').items() if v == mod]
    print(interest)
    X=nx.subgraph(G, interest)
    subgraphs['int'][IC]=X
    inmod_graph(X, 'interaction')

# Summary
Overall, within each module the interaction term is segregated. These are positive graphs, therefore they are either increasing or decreasing together. This indicates to me that the interaction is fairly unique and not densely correlated. It maybe that is it correlated more strongly to other modules. 

In [ ]:
nega_subgraphs={'BMI':{'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':{'IC12':{},'IC15':{}},
            'int':{'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}}

for IC, value in nega_graphs['int'].items():
    print(IC)
    G=value['graphs']
    mod=G.nodes(data=True)['int']['modules']
    print(mod)
    interest=[k for k,v in nx.get_node_attributes(G, 'modules').items() if v == mod]
    print(interest)
    X=nx.subgraph(G, interest)
    subgraphs['int'][IC]=X
    inmod_graph(X, 'interaction')

# Summary
we can see that the inverse correlations are overall much weaker than the positive. Further, the interaction term is positively correlated with BMI and head motion, but negatively correlated with age at onset of menses. This is expected since the relationship between BMI and age at onset of menses is inverse. (see R notebook)

## Creating module based graphs

In [ ]:
community_graphs={'BMI':{'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':{'IC12':{},'IC15':{}},
            'int':{'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}}
for IC, data in graphs['int'].items():
    comm_graph = community.induced_graph(data['modules']['partition'], 
                                    data['modules']['graph'])
    community_graphs['int'][IC]=comm_graph
    print(IC)
    module_fig(comm_graph, group, IC)

# Summary
The module graphs show that the connectivity of the modules containing the interaction term are rather weak. Especially in IC3 and IC12 (module 2), they are only weakly related to the other modules. In ICs 4 and 13 we see a more robust response. This suggest to me that 4 and 12 are driving their respective connections with 3 and 12. 

In [ ]:
nega_community_graphs={'BMI':{'IC1':{},'IC3':{},'IC4':{},'IC5':{},'IC6':{},'IC7':{},'IC8':{},'IC9':{},'IC10':{},'IC11':{},'IC12':{},'IC13':{},'IC14':{},'IC15':{}},
            'AoM':{'IC12':{},'IC15':{}},
            'int':{'IC2':{},'IC3':{},'IC6':{},'IC12':{},'IC13':{}}}
for IC, data in nega_graphs['int'].items():
    comm_graph = community.induced_graph(data['modules']['partition'], 
                                    data['modules']['graph'])
    community_graphs['int'][IC]=comm_graph
    print(IC)
    module_fig(comm_graph, group, IC)

# Summary
There are far fewer nodes and modules in the negative correlation graphs. Interestingly, in the IC2 and IC3 the module with the interaction term is different (0 vs. 2). It appears IC2 is driving the inverse relationship. 

In [ ]:
ALL_DATA={'interest':interest,
'dfs':dfs,
'zscores_thr':zscores_thr,
'graphs':graphs,
'nega_graphs':nega_graphs,
'subgraphs':subgraphs,
'community_graphs':community_graphs}

In [ ]:
pickle.dump(ALL_DATA, open('/Users/gracer/Google Drive/HCP_graph/1200/datasets/final.pkl', 'wb'), protocol=4)